In [66]:
import pandas as pd 
import numpy as np
from word2number import w2n

In [67]:
df_ejemplo=pd.read_csv("HR RAW DATA.csv",index_col=0)

In [68]:
#Homogeneizar nombre columnas 
nuevas_columnas = {columna: columna.lower() for columna in df_ejemplo.columns}

In [69]:
df_ejemplo.rename(columns=nuevas_columnas, inplace= True)

In [70]:
#Modificamos nombre de la columna para que sea mas intuitivo
df_ejemplo.rename(columns = {"datebirth": "yearbirth"}, inplace = True )

In [71]:
#Eliminamos las siguientes columnas:
#Role Department: Fusiona otras columnas, no le vemos sentdo
#Number Children: no hay valores
#Over 18: columna sin sentido.Esta desactualizada, se puede calcular con otras columnas
#Same As Month Income: Es una columna repetida, una copia de month income
#Employee Count: No tienen sentido. El recuento se puede hacer por filas, o por números de referncia de empleado
#Salary : la eliminamos y creamos una de nuevo con el cálculo de los datos 

In [72]:
df_ejemplo.drop(['roledepartament','numberchildren','over18', 'sameasmonthlyincome','employeecount','salary'],axis = 1, inplace=True)

In [73]:
#lista con columnas a limpiar
columnas_limpiar = ['dailyrate','employeenumber','monthlyincome','performancerating','totalworkingyears','worklifebalance','distancefromhome']

In [74]:
#Para columna de Age

def convertir_a_numero(valor):
    if isinstance(valor, int): # Verifica si el valor ya es un entero, si lo es no hace nada
        return valor
    elif isinstance(valor, str): # Intenta convertir el valor a número
        try: 
            numero = int(valor)
            return numero
        except ValueError:
            try: # aquí es donde se usa la función w2n en el try pq si la conversion falla entonces se usará
                numero = w2n.word_to_num(valor)
                return numero
            except ValueError:
                print(f"Error: No se pudo convertir '{valor}' a número.")
                return None
    else:
        print(f"Error: Tipo de dato no soportado para el valor '{valor}'.")
        return None


In [75]:
df_ejemplo['age'] = df_ejemplo['age'].apply(convertir_a_numero) #llamamos a la función

In [76]:
def limpiar_datos (columna):

      try:
          df_ejemplo[columna] =(df_ejemplo[columna].str.replace('$', ''))
    
      except:
          pass
      
      try:
          df_ejemplo[columna] =(df_ejemplo[columna].str.replace(',', '.')) 
    
      except:
          pass
      try:
          
         df_ejemplo[columna] = df_ejemplo[columna].abs()

      except:
          pass
      
      try:
        df_ejemplo[columna]=pd.to_numeric(df_ejemplo[columna]).astype(float)
      except:
        pass
      return df_ejemplo[columna]

In [77]:
#Limpiamos las columnas de numeros para convertirlas a numericas
for i in columnas_limpiar:
   df_ejemplo[i]=limpiar_datos(i)
   print(df_ejemplo[i])
   
 

0        684.0
1        699.0
2        532.0
3        359.0
4       1319.0
         ...  
1609     530.0
1610     805.0
1611     903.0
1612    1229.0
1613     566.0
Name: dailyrate, Length: 1614, dtype: object
0       162.0
1       259.0
2       319.0
3         NaN
4         NaN
        ...  
1609    967.0
1610    972.0
1611      NaN
1612    990.0
1613      NaN
Name: employeenumber, Length: 1614, dtype: float64
0       19537.0
1       19999.0
2       19232.0
3       17169.0
4           NaN
         ...   
1609    10325.0
1610     4447.0
1611        NaN
1612        NaN
1613    10845.0
Name: monthlyincome, Length: 1614, dtype: float64
0       3.0
1       3.0
2       3.0
3       3.0
4       3.0
       ... 
1609    NaN
1610    3.0
1611    3.0
1612    NaN
1613    3.0
Name: performancerating, Length: 1614, dtype: float64
0        NaN
1       34.0
2       22.0
3        NaN
4        NaN
        ... 
1609     NaN
1610     NaN
1611     9.0
1612    12.0
1613     NaN
Name: totalworkingyears, Lengt

In [78]:
#

_____

- Cambiamos de categoria la columna 'género'. Reemplazamos sus valores de 0 y 1 por Male y Famele.

- Homogeneizamos la columna 'maritalstatus'

- Modificamos los valores de la columna  'remotework', 0 y 1, True or False por Yes y No

- Al analizar la columna de employee number, encontramos muchos valores duplicados y la unica diferencia entre estos duplicados eran los valores que se encuentran en la columna 'remotework'. Allí detectamos que la equivalencia era 0 = 'No' y 1 = 'Yes'


In [79]:
df_ejemplo['gender'] = df_ejemplo['gender'].replace({0: 'male', 1: 'female'})
df_ejemplo['gender'].value_counts()

gender
male      971
female    643
Name: count, dtype: int64

In [81]:
df_ejemplo['maritalstatus']= df_ejemplo['maritalstatus'].replace({'divorced':'Divorced','Marreid':'Married'})

In [84]:
df_ejemplo['remotework'].value_counts()

remotework
1        360
True     345
0        309
False    305
Yes      295
Name: count, dtype: int64

In [93]:
df_ejemplo['remotework']= df_ejemplo['remotework'].replace({('1'):'Yes',('0') or ('False'): 'No', 'True': 'Yes'})

______

COLUMNA 13. HourlyRate: La tarifa por hora del empleado.

- NO hay nulos
- Pasamos de srt a int. 
- La media cobra por hora 62.59 

In [ ]:
df_ejemplo['HourlyRate'].isnull().sum()

0

In [ ]:
# Pasamos el valor a integer. 
# ponemos errors='coerce' ya que hay valores que no se puede transforar, como es el caso de 'Not Available'. Se convertiá a NaN

df_ejemplo['HourlyRate'] = pd.to_numeric(df_ejemplo['HourlyRate'], errors='coerce')

# Convertir la columna a tipo entero, tratando los NaN como 0
df_ejemplo['HourlyRate'] = df_ejemplo['HourlyRate'].fillna(0).astype(int)

In [ ]:
df_ejemplo['HourlyRate'].isnull().sum()

0

In [ ]:
df_ejemplo['HourlyRate']

0       51
1       65
2       58
3       82
4       45
        ..
1609    51
1610    57
1611    41
1612    84
1613    75
Name: HourlyRate, Length: 1614, dtype: int64

In [ ]:
df_ejemplo['HourlyRate'].describe()

count    1614.000000
mean       62.591698
std        24.525022
min         0.000000
25%        46.000000
50%        64.000000
75%        83.000000
max       100.000000
Name: HourlyRate, dtype: float64

____

COLUMNA 14. JobInvolvement: Nivel de implicación del empleado en su trabajo.
- Que significa? Quien lo valora? El mánager/jefe? Es una auto-evaluación?
- Se piede relacionar con otras columnas? Ej: salario / número de horas que se trabajan, etc...

In [ ]:
df_ejemplo['JobInvolvement'].value_counts()

JobInvolvement
3    955
2    406
4    164
1     89
Name: count, dtype: int64

In [ ]:
df_ejemplo['JobInvolvement'].size

1614

In [ ]:
df_ejemplo['JobInvolvement'].isnull().sum()

0

______

COLUMNA 15: JobLevel - Nivel Jerárquico del empleado en la empresa

-tipo de dato INT

-No hay valores nulos

In [ ]:
df_ejemplo['JobLevel'].unique()

array([5, 4, 3, 2, 1])

In [ ]:
df_ejemplo['JobLevel'].isnull().sum()

0

COLUMNA 16: JobRole -El rol o puesto de trabajo del empleado.

-No hay nulos

-Tipo de dato 'object'

-Cambie la infromacion a letras minusculas (estaba en formato messenger)

In [ ]:
df_ejemplo['JobRole']= [elemento.lower() for elemento in df_ejemplo['JobRole']]

In [ ]:
df_ejemplo['JobRole'].value_counts()

COLUMNA 17: JobSatisfaction - Nivel de satisfacción del empleado con su trabajo.

-Tipo de dato: int64

-No hay valores nulos

-Dejo registrada la media de satisfaccion laboral

In [ ]:
df_ejemplo['JobSatisfaction'].unique()

array([3, 4, 1, 2])

In [ ]:
df_ejemplo['JobSatisfaction'].value_counts()

4    514
3    481
1    317
2    302
Name: JobSatisfaction, dtype: int64

In [ ]:
df_ejemplo['JobSatisfaction'].describe(include='object')

count    1614.000000
mean        2.738538
std         1.106163
min         1.000000
25%         2.000000
50%         3.000000
75%         4.000000
max         4.000000
Name: JobSatisfaction, dtype: float64

COLUMNA 18: MaritalStatus - El estado civil del empleado (por ejemplo, "Single", "Married", etc.).

-Tipo de dato: 

-Datos nulos: 651. NaN pueden ser floats o str 

-Tenemos Divorce escrito de dos formas diferentes. Divorced/divorced. Lo unifico en Divorced
Ver si lo modificamos todo a minusculas
-Married idem anterior

In [ ]:
df_ejemplo['MaritalStatus'].isnull().sum()

651

In [ ]:
df_ejemplo['MaritalStatus']= df_ejemplo['MaritalStatus'].replace('divorced', 'Divorced')

In [ ]:
df_ejemplo['MaritalStatus']= df_ejemplo['MaritalStatus'].replace('Marreid', 'Married')

In [ ]:
df_ejemplo['MaritalStatus'].value_counts()

Married     439
Single      325
Divorced    199
Name: MaritalStatus, dtype: int64

COLUMNA 19: MonthlyIncome - Ingresos mensuales del empleado.

-Datos nulos: 843

-Tipo de dato: object

In [ ]:
df_ejemplo['MonthlyIncome'].isnull().sum()

843

In [ ]:
df_ejemplo["MonthlyIncome"]= df_ejemplo["MonthlyIncome"].str.replace(',','.')

In [ ]:
df_ejemplo["MonthlyIncome"]=pd.to_numeric(df_ejemplo["MonthlyIncome"]).astype(float)

COLUMNA 20: MonthlyRate - Tasa mensual del empleado.

 _ significa la tasa mensual de salario o sueldo que recibe un empleado por su trabajo en una empresa. Es el monto específico que se paga a los empleados en forma regular cada mes. Esta información se suele registrar en una tabla de empleados para llevar un registro de los salarios y compensaciones que se les otorgan. La tasa mensual puede variar dependiendo del puesto, la experiencia, las habilidades y otros factores relevantes.

-tipo de dato: int 

-No hay nulos

Hay mucha diferencia entre el minimo y el máximo

In [ ]:
df_ejemplo['MonthlyRate'].describe()

count     1614.000000
mean     14284.495663
std       7110.414585
min       2094.000000
25%       8001.000000
50%      14248.500000
75%      20364.000000
max      26999.000000
Name: MonthlyRate, dtype: float64

COLUMNA 21: NUMCOMPANIESWORKED - Número de compañías en las que el empleado ha trabajado.

-tipo de dato: Int

-No hay nulos 

-Cambiamos el tamaño del nombre de la columna, mismo patron que el resto de las columnas

In [ ]:
df_ejemplo['NUMCOMPANIESWORKED'].unique()

array([7, 0, 1, 3, 2, 4, 8, 9, 5, 6])

In [ ]:
df_ejemplo['NUMCOMPANIESWORKED'].value_counts()

1    573
0    226
3    169
4    157
2    156
7     84
6     73
5     66
9     59
8     51
Name: NUMCOMPANIESWORKED, dtype: int64

Over18 - Indica si el empleado es mayor de 18 años.

-valores nulos: 901

-Esta columna no tiene sentido


In [ ]:
df_ejemplo['Over18'].isnull().sum()

901

In [ ]:
df_ejemplo['Over18'].unique()

array(['Y', nan], dtype=object)

______

COLUMNA 23: OverTime

solo 3 tipos de valores, 40% Nan

In [ ]:
df_ejemplo['overtime'].unique()

array(['No', nan, 'Yes'], dtype=object)

In [ ]:
df_ejemplo['overtime'].value_counts()

No     682
Yes    256
Name: overtime, dtype: int64

In [ ]:
df_ejemplo['OverTime'].isnull().sum()

In [ ]:
df_ejemplo.info()

COLUMNA 24: PercentSalaryHike	

muchos porcentajes, ningún null

In [ ]:
df_ejemplo['percentsalaryhike'].unique()

array([13, 14, 11, 19, 12, 25, 16, 17, 22, 23, 20, 15, 21, 24, 18])

In [ ]:
df_ejemplo['percentsalaryhike'].value_counts()

11    232
13    230
12    225
14    220
15    110
18     98
17     88
16     86
19     82
20     60
22     59
21     51
23     29
24     25
25     19
Name: percentsalaryhike, dtype: int64

In [ ]:
df_ejemplo['PercentSalaryHike'].describe()

In [ ]:
df_ejemplo['PercentSalaryHike'].isnull().sum()

COLUMNA 25: PerformanceRating

Pocos null, hay que cambiar los float por int, solo valores 3 y 4

In [ ]:
df_ejemplo['PerformanceRating'].unique()

In [ ]:
df_ejemplo['PerformanceRating'].value_counts()

In [ ]:
df_ejemplo['PerformanceRating'].describe()

In [ ]:
df_ejemplo['PerformanceRating']=df_ejemplo['PerformanceRating'].str.replace(',','.')
df_ejemplo['PerformanceRating']=df_ejemplo['PerformanceRating'].astype(float)

In [ ]:
df_ejemplo['PerformanceRating'].describe()

In [ ]:
df_ejemplo['PerformanceRating'].isnull().sum()

COLUMNA 26: RelationshipSatisfaction

este aprece que está bien

In [ ]:
df_ejemplo['RelationshipSatisfaction'].unique()

In [ ]:
df_ejemplo['relationshipsatisfaction'].value_counts()

3    504
4    468
2    339
1    303
Name: relationshipsatisfaction, dtype: int64

In [ ]:
df_ejemplo['RelationshipSatisfaction'].describe()

In [ ]:
df_ejemplo['RelationshipSatisfaction'].isnull().sum()

COLUMNA 27: StandardHours	

solo 80 y nan. Podemos eliminarla

In [ ]:
df_ejemplo['StandardHours'].unique()

In [ ]:
df_ejemplo['StandardHours'].isnull().sum()

COLUMNA 28: StockOptionLevel

podríamos cambiar el nombre de la columna para que sea mas entendible

In [ ]:
df_ejemplo['StockOptionLevel'].unique()

In [ ]:
df_ejemplo['stockoptionlevel'].value_counts()

0    687
1    666
2    172
3     89
Name: stockoptionlevel, dtype: int64

In [ ]:
df_ejemplo['StockOptionLevel'].describe()

In [ ]:
df_ejemplo['StockOptionLevel'].isnull().sum()

COLUMNA 29: TOTALWORKINGYEARS

hay que cambiar el nombre de la col por minus o title, cambiar los float por int

In [ ]:
df_ejemplo['TOTALWORKINGYEARS'].unique()

In [ ]:
df_ejemplo['totalworkingyears'].describe()

count     1088
unique      40
top       10,0
freq       144
Name: totalworkingyears, dtype: object

In [ ]:
df_ejemplo['TOTALWORKINGYEARS']=df_ejemplo['TOTALWORKINGYEARS'].str.replace(',','.')
df_ejemplo['TOTALWORKINGYEARS']=df_ejemplo['TOTALWORKINGYEARS'].astype(float)

In [ ]:
df_ejemplo['TOTALWORKINGYEARS'].describe()

In [ ]:
df_ejemplo['TOTALWORKINGYEARS'].isnull().sum()

In [ ]:
df_ejemplo.rename(columns={'TOTALWORKINGYEARS':'TotalWorkingYears'})

COLUMNA 30: TrainingTimesLastYear

Todo parece coherente

In [ ]:
df_ejemplo['TrainingTimesLastYear'].unique()

In [ ]:
df_ejemplo['trainingtimeslastyear'].value_counts()

2    598
3    534
4    137
5    136
1     77
6     72
0     60
Name: trainingtimeslastyear, dtype: int64

In [ ]:
df_ejemplo['TrainingTimesLastYear'].isnull().sum()

______

Columna "YearsSinceLastPromotion"

1. no hay nulos y una columna de integers

In [ ]:
df_ejemplo["YearsSinceLastPromotion"].isnull().sum()

df_ejemplo["YearsSinceLastPromotion"].info()

<class 'pandas.core.series.Series'>
Index: 1614 entries, 0 to 1613
Series name: YearsSinceLastPromotion
Non-Null Count  Dtype
--------------  -----
1614 non-null   int64
dtypes: int64(1)
memory usage: 25.2 KB


Columna "YEARSWITHCURRMANAGER"

In [ ]:
df_ejemplo["YEARSWITHCURRMANAGER"].info()

<class 'pandas.core.series.Series'>
Index: 1614 entries, 0 to 1613
Series name: YEARSWITHCURRMANAGER
Non-Null Count  Dtype
--------------  -----
1614 non-null   int64
dtypes: int64(1)
memory usage: 25.2 KB


Columna "SameAsMonthlyIncome"

1. En principio es object y convertirla a float
2. Además es una columna con nulos, representan un 52% (nulos total 843)
3. compararla con "MonthlyIncome"
4. a eliminar pq es lo mismo que en MonthlyIncome

In [ ]:
df_ejemplo["SameAsMonthlyIncome"].info()

<class 'pandas.core.series.Series'>
Index: 1614 entries, 0 to 1613
Series name: SameAsMonthlyIncome
Non-Null Count  Dtype 
--------------  ----- 
771 non-null    object
dtypes: object(1)
memory usage: 25.2+ KB


In [ ]:
df_ejemplo['SameAsMonthlyIncome'] = df_ejemplo['SameAsMonthlyIncome'].replace(',', '.') #para poder convertir de obj a float primero hay 
                                                                                        #q sustituir las comas por puntos. Los float van con puntos
df_ejemplo['SameAsMonthlyIncome'] = df_ejemplo['SameAsMonthlyIncome'].astype(float) #convertimos a float
df_ejemplo['SameAsMonthlyIncome']

ValueError: could not convert string to float: '19537,0'

Columna "DateBirth"

1. Cambiar nombre de la columna a "Yearbirth"



In [ ]:
df_ejemplo["DateBirth"].info()

<class 'pandas.core.series.Series'>
Index: 1614 entries, 0 to 1613
Series name: DateBirth
Non-Null Count  Dtype
--------------  -----
1614 non-null   int64
dtypes: int64(1)
memory usage: 89.8 KB


Columna "Salary"

1. object convertirlo a float
2. eliminar el signo de dólares (referenciarlo en el nombre de la columna)
3. solo hay un elemento en la columna - no tiene mucho sentido candidata a ser eliminada y rehacerla con calculos 


In [ ]:
df_ejemplo["Salary"].info()

<class 'pandas.core.series.Series'>
Index: 1614 entries, 0 to 1613
Series name: Salary
Non-Null Count  Dtype 
--------------  ----- 
1614 non-null   object
dtypes: object(1)
memory usage: 89.8+ KB


In [ ]:
df_ejemplo["Salary"].nunique()

1

In [ ]:
df_ejemplo['Salary'] = df_ejemplo['Salary'].str.replace('$', '') #sacamos el símbolo de la currency de la columna para poder convertir a float

# Convertir los valores a tipo float 
df_ejemplo['Salary'] = df_ejemplo['Salary'].astype(float)

Columna "RoleDepartament"

1. % elevado de null un 81%, candidata a ser eliminada


In [ ]:
df_ejemplo["RoleDepartament"].isnull().sum()

1312

In [ ]:
df_ejemplo["RoleDepartament"].unique()

array([nan, ' ManaGER  -  Research & Development ',
       ' MANAger  -  Research & Development ',
       ' heaLtHcArE repResENtATiVe  -  Research & Development ',
       ' saLEs exEcutIVe  -  Sales ', ' Sales ExECuTIVe  -  Sales ',
       ' heALthCArE RePResEntaTIVe  -  Research & Development ',
       ' lABORAtOry tECHnIcIan  -  Research & Development ',
       ' ManufactUrINg DiReCTOr  -  Research & Development ',
       ' SaleS eXEcUtIVE  -  Sales ',
       ' HEaLthcaRe rEprEsEnTAtiVe  -  Research & Development ',
       ' rESeArCH sCientIST  -  Research & Development ',
       ' healTHCAre rePREseNtATiVE  -  Research & Development ',
       ' mANaGer  -  Research & Development ',
       ' HEAltHCaRe REPreseNTatIve  -  Research & Development ',
       ' SaLeS eXECUTivE  -  Sales ',
       ' reSeARCH SCIEnTiST  -  Research & Development ',
       ' ReSearch DiRecTOR  -  Research & Development ',
       ' resEaRCh SciEnTist  -  Research & Development ',
       ' hUmaN REsoUrCes  -  H

In [ ]:
df_ejemplo["RoleDepartament"].value_counts()

RoleDepartament
 MaNAgeR  -  Sales                                        2
 ManaGER  -  Research & Development                       1
 ReseaRch scIENTisT  -  Research & Development            1
 ManufacTURInG DIRECtOR  -  Research & Development        1
 hEalthCaRe reprEseNTaTiVe  -  Research & Development     1
                                                         ..
 saLES eXEcUTiVE  -  Sales                                1
 mANUfacTURiNG dIRectOR  -  Research & Development        1
 huMAn ResOurces  -  Human Resources                      1
 HUMAN ResoURCeS  -  Human Resources                      1
 sAleS EXECUtIvE  -  Sales                                1
Name: count, Length: 301, dtype: int64

Columna "NUMBERCHILDREN"

1. no hay valores, mejor eliminarla

In [ ]:
df_ejemplo["NUMBERCHILDREN"].info()

<class 'pandas.core.series.Series'>
Index: 1614 entries, 0 to 1613
Series name: NUMBERCHILDREN
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 89.8 KB


Columna "RemoteWork"

1. hay 5 valores unicos: 0,1,yes,false,true; no sabemos qué es 0 y 1


In [ ]:
df_ejemplo["RemoteWork"].value_counts()


RemoteWork
1        360
True     345
0        309
False    305
Yes      295
Name: count, dtype: int64

In [ ]:
df_ejemplo["RemoteWork"].info()

<class 'pandas.core.series.Series'>
Index: 1614 entries, 0 to 1613
Series name: RemoteWork
Non-Null Count  Dtype 
--------------  ----- 
1614 non-null   object
dtypes: object(1)
memory usage: 25.2+ KB


COLUMNA WORKLIFEBALANCE

In [ ]:
df_ejemplo["WORKLIFEBALANCE"].info()

<class 'pandas.core.series.Series'>
Index: 1614 entries, 0 to 1613
Series name: WORKLIFEBALANCE
Non-Null Count  Dtype 
--------------  ----- 
1506 non-null   object
dtypes: object(1)
memory usage: 25.2+ KB


In [ ]:
df_ejemplo["WORKLIFEBALANCE"].value_counts()

WORKLIFEBALANCE
3,0    913
2,0    359
4,0    155
1,0     79
Name: count, dtype: int64

In [ ]:
df_ejemplo.info()

In [ ]:
df_ejemplo["WORKLIFEBALANCE"].info()

In [ ]:
df_ejemplo["WORKLIFEBALANCE"].isnull().sum()

108

In [ ]:
df_ejemplo['WORKLIFEBALANCE'] = df_ejemplo['WORKLIFEBALANCE'].str.replace(',', '.') #sacamos el símbolo de la currency de la columna para poder convertir a float

# Convertir los valores a tipo float 
df_ejemplo['WORKLIFEBALANCE'] = df_ejemplo['WORKLIFEBALANCE'].astype(float)

Columna YearsAtCompany

In [ ]:
df_ejemplo["YearsAtCompany"].info()

<class 'pandas.core.series.Series'>
Index: 1614 entries, 0 to 1613
Series name: YearsAtCompany
Non-Null Count  Dtype
--------------  -----
1614 non-null   int64
dtypes: int64(1)
memory usage: 25.2 KB


In [ ]:
df_ejemplo["YearsAtCompany"].value_counts()

YearsAtCompany
5     208
1     171
3     141
2     141
10    133
7     115
4     114
8     106
9      94
6      78
0      44
11     36
20     29
13     26
15     21
14     19
22     17
12     15
18     15
16     14
21     14
19     12
17      9
24      7
25      5
33      5
26      4
32      3
27      3
31      3
36      3
29      2
23      2
40      2
34      1
37      1
30      1
Name: count, dtype: int64

Columna YearsInCurrentRole
1. muchos nulos quizás conveniente eliminarla

In [ ]:
df_ejemplo["YearsInCurrentRole"].info()


<class 'pandas.core.series.Series'>
Index: 1614 entries, 0 to 1613
Series name: YearsInCurrentRole
Non-Null Count  Dtype 
--------------  ----- 
34 non-null     object
dtypes: object(1)
memory usage: 25.2+ KB


In [ ]:
df_ejemplo["YearsInCurrentRole"].isnull().sum()

1580